In [1]:
import pandas as pd
from numpy import *
import glob

import plotly.graph_objects as go

## not used in current version - use pip install circle-fitting-3d to install circle-fitting-3d by CristianoPizzamiglio - https://github.com/CristianoPizzamiglio/circle-fitting-3d

In [2]:
## codes to compute the best fit circle in 3D

#-------------------------------------------------------------------------------
# FIT CIRCLE 2D
# - Find center [xc, yc] and radius r of circle fitting to set of 2D points
# - Optionally specify weights for points
#
# - Implicit circle function:
#   (x-xc)^2 + (y-yc)^2 = r^2
#   (2*xc)*x + (2*yc)*y + (r^2-xc^2-yc^2) = x^2+y^2
#   c[0]*x + c[1]*y + c[2] = x^2+y^2
#
# - Solution by method of least squares:
#   A*c = b, c' = argmin(||A*c - b||^2)
#   A = [x y 1], b = [x^2+y^2]
#-------------------------------------------------------------------------------
def fit_circle_2d(x, y, w=[]):
    
    A = array([x, y, ones(len(x))]).T
    b = x**2 + y**2
    
    # Modify A,b for weighted least squares
    if len(w) == len(x):
        W = diag(w)
        A = dot(W,A)
        b = dot(W,b)
    
    # Solve by method of least squares
    c = linalg.lstsq(A,b,rcond=None)[0]
    
    # Get circle parameters from solution c
    xc = c[0]/2
    yc = c[1]/2
    r = sqrt(c[2] + xc**2 + yc**2)
    return xc, yc, r


#-------------------------------------------------------------------------------
# RODRIGUES ROTATION
# - Rotate given points based on a starting and ending vector
# - Axis k and angle of rotation theta given by vectors n0,n1
#   P_rot = P*cos(theta) + (k x P)*sin(theta) + k*<k,P>*(1-cos(theta))
#-------------------------------------------------------------------------------
def rodrigues_rot(P, n0, n1):
    
    # If P is only 1d array (coords of single point), fix it to be matrix
    if P.ndim == 1:
        P = P[newaxis,:]
    
    # Get vector of rotation k and angle theta
    n0 = n0/linalg.norm(n0)
    n1 = n1/linalg.norm(n1)
    k = cross(n0,n1)
    k = k/linalg.norm(k)
    theta = arccos(dot(n0,n1))
    
    # Compute rotated points
    P_rot = zeros((len(P),3))
    for i in range(len(P)):
        P_rot[i] = P[i]*cos(theta) + cross(k,P[i])*sin(theta) + k*dot(k,P[i])*(1-cos(theta))

    return P_rot


#-------------------------------------------------------------------------------
# ANGLE BETWEEN
# - Get angle between vectors u,v with sign based on plane with unit normal n
#-------------------------------------------------------------------------------
def angle_between(u, v, n=None):
    if n is None:
        return arctan2(linalg.norm(cross(u,v)), dot(u,v))
    else:
        return arctan2(dot(n,cross(u,v)), dot(u,v))

    
#-------------------------------------------------------------------------------
# - Make axes of 3D plot to have equal scales
# - This is a workaround to Matplotlib's set_aspect('equal') and axis('equal')
#   which were not working for 3D
#-------------------------------------------------------------------------------
def set_axes_equal_3d(ax):
    limits = array([ax.get_xlim3d(), ax.get_ylim3d(), ax.get_zlim3d()])
    spans = abs(limits[:,0] - limits[:,1])
    centers = mean(limits, axis=1)
    radius = 0.5 * max(spans)
    ax.set_xlim3d([centers[0]-radius, centers[0]+radius])
    ax.set_ylim3d([centers[1]-radius, centers[1]+radius])
    ax.set_zlim3d([centers[2]-radius, centers[2]+radius])

In [3]:
def fit_circle3D(P):
    # P is an array of points to which we must fit a circle
    #-------------------------------------------------------------------------------
    # (1) Fitting plane by SVD for the mean-centered data
    # Eq. of plane is <p,n> + d = 0, where p is a point on plane and n is normal vector
    #-------------------------------------------------------------------------------
    P_mean = P.mean(axis=0)
    P_centered = P - P_mean
    U,s,V = linalg.svd(P_centered)

    # Normal vector of fitting plane is given by 3rd column in V
    # Note linalg.svd returns V^T, so we need to select 3rd row from V^T
    normal = V[2,:]
    d = -dot(P_mean, normal)  # d = -<p,n>

    #-------------------------------------------------------------------------------
    # (2) Project points to coords X-Y in 2D plane
    #-------------------------------------------------------------------------------
    P_xy = rodrigues_rot(P_centered, normal, [0,0,1])

    # ax[0].scatter(P_xy[:,0], P_xy[:,1], alpha=alpha_pts, label='Projected points')

    #-------------------------------------------------------------------------------
    # (3) Fit circle in new 2D coords
    #-------------------------------------------------------------------------------
    xc, yc, r = fit_circle_2d(P_xy[:,0], P_xy[:,1])
    C = rodrigues_rot(array([xc,yc,0]), [0,0,1], normal) + P_mean
    C = C.flatten()

#     print('Fitting plane: n = %s' % array_str(normal, precision=4))
#     print('Fitting circle: center = %s, r = %.4g' % (array_str(C, precision=4), r))
    # print('Fitting arc: u = %s, θ = %.4g' % (array_str(u, precision=4), theta*180/pi))
    return(C, r)

## run the above codes for our data

In [4]:
files = glob.glob('./Data/Outputs/COM/*_smoothened_COM.csv')
files

['./Data/Outputs/COM\\2023-04-19_Trial7_Set1_5k_smoothened_COM.csv',
 './Data/Outputs/COM\\2023-04-21_Trial6_1k_smoothened_COM.csv',
 './Data/Outputs/COM\\2023-04-21_Trial7_1k_smoothened_COM.csv',
 './Data/Outputs/COM\\2023-04-28_Trial4_5k_smoothened_COM.csv',
 './Data/Outputs/COM\\2023-04-28_Trial5_5k_smoothened_COM.csv',
 './Data/Outputs/COM\\2023-04-28_Trial8_1k_smoothened_COM.csv']

In [5]:
info = pd.read_csv('./Data/Outputs/Raw_xyz_points/metadata.csv')
info

,filename,start,end,framerate,chase_type,Notes
0,2023-04-19_Trial7_Set1_5k_xyzpts.csv,487.0,926.0,5000,Intact male chasing female,Perfect
1,2023-04-19_Trial8_Set1_5k_xyzpts.csv,NaN,NaN,5000,Intact male chasing male,80% digitized. Needs to fill some gaps
2,2023-04-21_Trial5_1k_xyzpts.csv,115.0,283.0,1000,Intact male chasing ablated male,Can't find the project file. Only the xyz pts ...
3,2023-04-21_Trial6_1k_xyzpts.csv,56.0,276.0,1000,Intact male chasing intact male,Perfect
4,2023-04-21_Trial7_1k_xyzpts.csv,112.0,703.0,1000,Intact male chasing intact male,Perfect
5,2023-04-28_Trial4_5k_xyzpts.csv,332.0,843.0,5000,Ablated male chasing intact male,Perfect
6,2023-04-28_Trial5_5k_xyzpts.csv,1420.0,2510.0,5000,Ablated male chasing intact male,Perfect
7,2023-04-28_Trial6_5k_xyzpts.csv,58.0,1164.0,5000,Ablated male chasing intact male,Can't find the project file. Only the xyz pts ...
8,2023-04-28_Trial8_1k_xyzpts.csv,108.0,276.0,1000,Intact male chasing ablated male,Perfect


In [8]:
for f in files:
    df = pd.read_csv(f)
    
    chasee_COM = df.loc[:, ['chasee_com_x', 'chasee_com_y', 'chasee_com_z']]
    chaser_COM = df.loc[:, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
    
    print(chasee_COM.isnull().values.any())
    print(chaser_COM.isnull().values.any())
    
    tt_r = []
    tt_c = []
    length_of_arc = 7 # number of frames used to compute the length of arch
    for idx, cc in enumerate([chasee_COM, chaser_COM]):
        test_c = zeros((len(cc), 3))
        test_r = zeros(len(cc))

        for ind in range(len(cc)-length_of_arc):
            P = cc.loc[ind:ind+length_of_arc, :].values*10e4
            test_c[ind],test_r[ind] = fit_circle3D(P)

        test_r[len(cc)-length_of_arc:len(cc)] = nan
        tt_r.append(test_r)
        tt_c.append(test_c)
        
    df['chasee_Trajec_r'] = tt_r[0]
    df['chasee_Trajec_curv'] = 1/tt_r[0]
    df['chaser_Trajec_r'] = tt_r[1]
    df['chaser_Trajec_curv'] = 1/tt_r[1]
    
    name = f.split('\\')[-1][:-8]
    df.to_csv('./Data/Outputs/curvature/' + name + '_len_' + str(length_of_arc) + '_curvature.csv')

False
False
False
False
False
False
False
False
False
False
False
False


In [9]:
tt_r[0]

array([35483.99500757, 32076.12666774, 28504.56144275, 22976.07627727,
       19927.03423513, 15705.19350258, 14307.02687129, 14881.05629527,
       15851.41306635, 17188.76899197, 21957.71328237, 26790.77243786,
       26626.69599003, 27886.87575127, 28742.80408654, 25345.43130826,
       23127.75087997, 25065.54554371, 31834.0473818 , 27194.15021802,
       21651.41690475, 23292.11286901, 28235.89209616, 34176.46331593,
       26816.79977603, 23500.08909151, 22047.12323478, 22857.83561325,
       33590.8253749 , 38175.58945335, 39000.06542582, 30829.59373735,
       26085.92771859, 22744.93262831, 21468.63707557, 21459.08344414,
       24610.3822137 , 30040.94422475, 34068.31145913, 39656.24045761,
       42849.95880567, 44712.98355468, 34693.72733898, 34949.001481  ,
       39381.84895221, 52479.18182422, 39535.42723484, 27712.82062326,
       19338.21444875, 15657.92988702, 15311.65488447, 16262.15199102,
       18704.79658655, 19484.58301414, 18733.48361722, 17278.35068396,
      

In [10]:
1/tt_r[0]

array([2.81817197e-05, 3.11758340e-05, 3.50821044e-05, 4.35235324e-05,
       5.01830823e-05, 6.36732047e-05, 6.98957239e-05, 6.71995307e-05,
       6.30858584e-05, 5.81775228e-05, 4.55420830e-05, 3.73262847e-05,
       3.75562931e-05, 3.58591622e-05, 3.47913167e-05, 3.94548425e-05,
       4.32380998e-05, 3.98954014e-05, 3.14129079e-05, 3.67726144e-05,
       4.61863537e-05, 4.29329879e-05, 3.54159166e-05, 2.92599030e-05,
       3.72900573e-05, 4.25530302e-05, 4.53573915e-05, 4.37486741e-05,
       2.97700336e-05, 2.61947494e-05, 2.56409826e-05, 3.24363665e-05,
       3.83348452e-05, 4.39658370e-05, 4.65795754e-05, 4.66003128e-05,
       4.06332576e-05, 3.32879018e-05, 2.93527902e-05, 2.52167121e-05,
       2.33372453e-05, 2.23648686e-05, 2.88236542e-05, 2.86131208e-05,
       2.53924086e-05, 1.90551751e-05, 2.52937699e-05, 3.60843818e-05,
       5.17110824e-05, 6.38654028e-05, 6.53097270e-05, 6.14924766e-05,
       5.34622227e-05, 5.13226277e-05, 5.33803547e-05, 5.78758944e-05,
      